# BibTeX to Markdown Converter

This program converts a BibTeX data file into a formatted paper list in Markdown. BibTeX is widely used for referencing in academic papers, but Markdown is a simpler, more readable format for displaying lists of references on web pages or in documents. By converting BibTeX entries to Markdown, you can easily present bibliographic information in a clean way.

### Example
Given the following BibTeX entry:
```bibtex
@article{karpathy2015visualizing,
  title={Visualizing and understanding recurrent networks},
  author={Karpathy, Andrej and Johnson, Justin and Li, Fei-Fei},
  journal={arXiv preprint arXiv:1506.02078},
  year={2015}
}
```
The program will produce the following Markdown formatted paper list (or adjust format in the generate_markdown() function):
**Visualizing and understanding recurrent networks**. Andrej Karpathy, Justin Johnson, and Fei-Fei Li. *arXiv preprint arXiv:1506.02078* (2015) [[link]](https://arxiv.org/pdf/1506.02078))

Get an API key from the links below:\
https://serpdog.io/google-scholar-api/ (1,000 free searches a month) \
https://serpapi.com/google-scholar-organic-results (100 free searches a month) \
https://www.scaleserp.com/ (100 free searches a month) \
https://www.scraperapi.com/blog/best-google-scholar-apis-proxies/

In [ ]:
import requests
import bibtexparser

In [27]:
# https://docs.serpdog.io/google-scholar-api
def get_paper_link(title, authors):
    api_key = 'API'  # Replace with your Serpdog API key
    params = {
        "api_key": api_key,
        "q": f"{title} {authors}"
    }
    
    response = requests.get("https://api.serpdog.io/scholar", params=params)

    
    if response.status_code == 200:
        results = response.json().get("scholar_results", [])
        if results:
            # Assuming the first result is the most relevant one
            resource_link = results[0].get("resources", [])
            if resource_link:
                return resource_link[0].get("link", "No link found")
            else:
                return results[0].get("title_link", "No link found")
        else:
            return "No results found"
    else:
        return f"Error: {response.status_code}"

In [ ]:
# Extract bibtex entries from a file
def extract_bibtex_entries(file_path):
    with open(file_path) as bibtex_file:
        bib_database = bibtexparser.load(bibtex_file)
    return bib_database.entries


# Function to format authors
def format_authors(authors):
    author_list = authors.split(' and ')
    formatted_authors = []
    for author in author_list:
        names = author.split(', ')
        if len(names) == 2:
            formatted_authors.append(f"{names[1]} {names[0]}")
        else:
            formatted_authors.append(names[0])
    if len(formatted_authors) > 1:
        return ', '.join(formatted_authors[:-1]) + ', and ' + formatted_authors[-1]
    else:
        return formatted_authors[0]


# Generate markdown for the papers
def generate_markdown(papers):
    markdown = "# Survey Paper List\n\n"
    for i, paper in enumerate(papers[:5]):  # Remove 5 after the test run
        title = paper.get('title', 'No Title').strip('{}').replace('\n', ' ')
        authors = format_authors(paper.get('author', 'No Author'))
        venue = paper.get('booktitle', paper.get('journal', 'No venue'))
        year = paper.get('year', 'No Year')
        link = get_paper_link(title, authors)
        markdown += f"- **{title}.** {authors}. *{venue}* ({year}) [[link]]({link})\n"
        print(f"Index: {i}, title: {title}. link: {link}")
    return markdown


# Path to your .bib file
bib_file_path = 'authorship_survey.bib'

# Extract bibtex entries
papers = extract_bibtex_entries(bib_file_path)

# Generate markdown
markdown_content = generate_markdown(papers)

with open('paper_list.md', 'w') as f:
    f.write(markdown_content)

len(papers)